In [ ]:
#r "nuget:TileDB.CSharp"

## Create an array

In [ ]:

var ctx = new TileDB.Context();
var dom = new TileDB.Domain(ctx);
//add dimensions
dom.add_int32_dimension("rows",1,4,4);
dom.add_int32_dimension("cols",1,4,4);
var schema = new TileDB.ArraySchema(ctx,TileDB.ArrayType.TILEDB_DENSE);
schema.set_domain(dom);
//add attribute
var attr1 = TileDB.Attribute.create_attribute(ctx,"a",TileDB.DataType.TILEDB_INT32);
schema.add_attribute(attr1);
string array_uri = "test_metadata_array";
var vfs = new TileDB.VFS(ctx);
if(vfs.is_dir(array_uri))
{
    vfs.remove_dir(array_uri);

}

TileDB.Array.create(array_uri,schema);



In [ ]:
// use ArrayUtil to get array_schema json string
//var jsonstr = TileDB.ArrayUtil.get_array_schema_json_str(array_uri,ctx);
//System.Console.WriteLine("{0}",jsonstr);
var j=TileDB.CoreUtil.GetArraySchemaJson(array_uri,ctx);
System.Console.WriteLine("{0}",j.ToString());


## Add metadata

In [ ]:
// add int metadata
List<int> int_metadata = new List<int>() {1,100};
TileDB.CoreUtil.AddArrayMetadataByList<int>(array_uri,"int_metadata",int_metadata,ctx);
// add double metadata
var double_metadata = new List<double>() {1.0,100.0,1000.0};
TileDB.CoreUtil.AddArrayMetadataByList<double>(array_uri,"double_metadata", double_metadata,ctx);
// add string key value pair 
TileDB.CoreUtil.AddArrayMetadataByStringKeyValue(array_uri,"key1","value1",ctx);
// add multiple string key value pairs 
var key_value_map = new System.Collections.Generic.Dictionary<string,string>();
key_value_map["key2"] = "value2";
key_value_map["key3"] = "value3";
TileDB.CoreUtil.AddArrayMetadataByStringMap(array_uri,key_value_map,ctx);

## Get metadata

### get metadata from index

In [ ]:
// get metadata json string from index
//var metadata_jsonstr_0 = TileDB.ArrayUtil.get_array_metadata_json_str_from_index(array_uri,0,ctx);
//System.Console.WriteLine("{0}",metadata_jsonstr_0);
var metadata_json = TileDB.CoreUtil.GetArrayMetadataJsonFromIndex(array_uri,0,ctx);
System.Console.WriteLine("{0}", metadata_json.ToString());


### get metadata from key

In [ ]:
// get metadata json string for key
//var double_metadata_jsonstr = TileDB.ArrayUtil.get_array_metadata_json_str_for_key(array_uri,"double_metadata",ctx);
//System.Console.WriteLine("{0}",double_metadata_jsonstr);
var double_metadata = TileDB.CoreUtil.GetArrayMetadataJsonForKey(array_uri,"double_metadata",ctx);
System.Console.WriteLine("{0}", double_metadata.ToString());
var double_list = Newtonsoft.Json.JsonConvert.DeserializeObject<List<double>>(metadata_json["value"].ToString());
System.Console.WriteLine("type of double_list:{0}, value of double_list:{1}",double_list.GetType(),string.Join(",",double_list));

In [ ]:
// get string metadata
var string_metadata = TileDB.CoreUtil.GetArrayMetadataJsonForKey(array_uri,"key2",ctx);
System.Console.WriteLine("{0}", string_metadata.ToString());
// access key and value
System.Console.WriteLine("{0}:{1}",string_metadata["key"],string_metadata["value"]);


### get all metadata

In [ ]:
//get json string for all metadata
//var metadata_jsonstr = TileDB.ArrayUtil.get_array_metadata_json_str(array_uri,ctx);
//System.Console.WriteLine("{0}",metadata_jsonstr);
var metadata_json = TileDB.CoreUtil.GetArrayMetadataJson(array_uri,ctx);
System.Console.WriteLine("{0}", metadata_json.ToString());

## Clean up

In [ ]:
if(vfs.is_dir(array_uri))
{
    vfs.remove_dir(array_uri);
}